# Make a theta-square plot

This is a basic example to analyze some events and make a $\Theta^2$ plot

In [ ]:
%matplotlib inline

In [ ]:
from astropy import units as u
from astropy.coordinates.angle_utilities import angular_separation

import matplotlib.pyplot as plt
import numpy as np

from ctapipe.io import event_source
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from ctapipe.reco import HillasReconstructor
from ctapipe.image import hillas_parameters, tailcuts_clean
from ctapipe.utils import datasets

Get source events in MC dataset. Here we stop at 10 events, just to make this example run fast, but for real use, one would need more statistics. 

In [ ]:
filename = datasets.get_dataset_path("gamma_test_large.simtel.gz")
source = event_source(filename, allowed_tels={1, 2, 3, 4}, max_events=10)
reco = HillasReconstructor()
calib = CameraCalibrator()

In [ ]:
off_angles = []
for event in source:

    # calibrating the event
    calib.calibrate(event)
    
    hillas_params = {}
    # pointing direction of the telescopes
    point_azimuth = {}
    point_altitude = {}
    
    subarray = event.inst.subarray
    
    # get hillas params for each event in different telescopes
    for tel_id in event.dl0.tels_with_data:

        # telescope pointing direction
        point_azimuth[tel_id] = event.mc.tel[tel_id].azimuth_raw * u.rad
        point_altitude[tel_id] = event.mc.tel[tel_id].altitude_raw * u.rad
        #        print(point_azimuth,point_altitude)

        # Camera Geometry required for hillas parametrization
        camgeom = subarray.tel[tel_id].camera

        # note the [0] is for channel 0 which is high-gain channel
        image = event.dl1.tel[tel_id].image[0]

        # Cleaning  of the image
        cleaned_image = image
        # create a clean mask of pixels above the threshold
        cleanmask = tailcuts_clean(
            camgeom, image, picture_thresh=10, boundary_thresh=5
        )
        # set all rejected pixels to zero
        cleaned_image[~cleanmask] = 0

        # Calulate hillas parameters
        # It fails for empty pixels
        try:
            hillas_params[tel_id] = hillas_parameters(camgeom, cleaned_image)
        except:
            pass

    if len(hillas_params) < 2:
        continue

    reco_result = reco.predict(hillas_params, event.inst, point_altitude, point_azimuth)

    # get angular offset between reconstructed shower direction and MC
    # generated shower direction
    off_angle = angular_separation(event.mc.az, event.mc.alt, reco_result.az, reco_result.alt)

    # Appending all estimated off angles
    off_angles.append(off_angle.to(u.deg).value)

calculate theta square for angles which are not nan

In [ ]:
off_angles = np.array(off_angles)
thetasquare = off_angles[np.isfinite(off_angles)]**2

## Plot the results

In [ ]:
plt.hist(thetasquare, bins=10, range=[0,0.4])
plt.xlabel(r'$\theta^2$ (deg)')
plt.ylabel("# of events")
plt.show()

again, this plot is not beautiful since we have such low stats